In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
%cd /content/drive/MyDrive/WorkSpace/AI_Research/Emotion/code/

/content/drive/MyDrive/WorkSpace/AI_Research/Emotion/code


In [3]:
import sys
sys.path.append("/content/drive/MyDrive/WorkSpace/AI_Research/Emotion/code/libs/")

In [4]:
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import pandas as pd
# from lr_finder import LRFinder
from utils.processing import data_processing
from utils import get_models
from utils import optimizers
from utils.datasets import data_generator
from utils.processing import data_augmentation
from utils import predict
from utils import attention_module

import json
import numpy as np
import keras
import tensorflow as tf
from keras import layers, Input
from keras.utils import np_utils 

from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.applications import VGG16, ResNet50, VGG19, ResNet50V2, MobileNetV2, MobileNetV2
from tensorflow.keras.optimizers import Adam, Adamax
from keras.models import load_model, Model



/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1615: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1641: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,


## Load data

In [ ]:
# Get data
emotion = ["Angry" , "Disgust", "Fear", "Happy", "Sad", "Surprise" , "Neutral"  ]
PATH_FER = "/content/drive/MyDrive/WorkSpace/AI_Research/datasets/Fer/icml_face_data.csv"
data = pd.read_csv(PATH_FER)

In [ ]:
X, y = data_processing.get_fer(data)

In [ ]:
X = np.array(X)
X = X.reshape((X.shape[0], 48, 48, 1))

In [ ]:
X_train,y_train, X_test, y_test, X_val, y_val = data_processing.load_fer(data)

In [ ]:
# Offline augmentation
# datagen_val = data_augmentation.val_augmentation(X_val)


# X_train, y_train = data_augmentation.aug_offline(X_train, y_train, num_class = 1, number_aug = 7)
# X_train, y_train = data_augmentation.aug_offline(X_train, y_train, num_class = 5, number_aug =1)

# X_train, y_train = shuffle(X_train, y_train)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(28709, 48, 48, 1)
(3589, 48, 48, 1)
(3589, 48, 48, 1)


## Data generator

In [ ]:
default_ws_dir =  "/content/drive/MyDrive/WorkSpace/AI_Research/Emotion"
configfer = json.load(open(f"{default_ws_dir}/code/configs/ferplus_config.json"))
config_train = json.load(open(f"{default_ws_dir}/code/configs/wb_config.json"))

In [7]:
img_size =224

In [ ]:
params_train = {
          'config': configfer,
          'shuffle': True,
          'augmentation': True}
params_val = {
          'config': configfer,
          'shuffle': None,
          'augmentation': None}         

train_ds = data_generator.DataGenerator(X_train, y_train, **params_train ).to_dataset(data_type = "train")
val_ds = data_generator.DataGenerator(X_val, y_val, **params_val).to_dataset(data_type = "val")
test_ds = data_generator.DataGenerator(X_test, y_test, **params_val).to_dataset(data_type = "val", batch_size = 1)

In [ ]:
val_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))>

In [ ]:
batch1 , batch2 = val_ds.take(2)

In [ ]:
batch1

(<tf.Tensor: shape=(32, 224, 224, 3), dtype=float32, numpy=
 array([[[[0.99607843, 0.99607843, 0.99607843],
          [0.99607843, 0.99607843, 0.99607843],
          [0.99607843, 0.99607843, 0.99607843],
          ...,
          [0.60784316, 0.60784316, 0.60784316],
          [0.6313726 , 0.6313726 , 0.6313726 ],
          [0.6313726 , 0.6313726 , 0.6313726 ]],
 
         [[0.99607843, 0.99607843, 0.99607843],
          [0.99607843, 0.99607843, 0.99607843],
          [0.99607843, 0.99607843, 0.99607843],
          ...,
          [0.60784316, 0.60784316, 0.60784316],
          [0.6313726 , 0.6313726 , 0.6313726 ],
          [0.6313726 , 0.6313726 , 0.6313726 ]],
 
         [[0.99607843, 0.99607843, 0.99607843],
          [0.99607843, 0.99607843, 0.99607843],
          [0.99607843, 0.99607843, 0.99607843],
          ...,
          [0.6039216 , 0.6039216 , 0.6039216 ],
          [0.627451  , 0.627451  , 0.627451  ],
          [0.627451  , 0.627451  , 0.627451  ]],
 
         ...,
 
      

In [8]:
# Create input shape
img_size_target = img_size
img_input = Input(shape=(img_size_target, img_size_target, 1))
img_conc = layers.Concatenate()([img_input, img_input, img_input])

In [ ]:
model = get_models.Pre_train_model(VGG16, None, (224,224,3))

# model.load_weights("/content/drive/MyDrive/WorkSpace/AI_Research/Emotion/checkpoints/FerPlusV11.hdf5")
model.compile(loss =config_train["loss"], optimizer =Adam(0.0001), metrics=[config_train["metric"]])
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=tf.math.exp(-0.1),
                              patience=3, min_lr=0 , verbose =1)

files = configfer["file_checkpoint"]
checkpoint_filepath = (f"{default_ws_dir}/checkpoints/{files}")

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose= 1,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=True, save_weights_only = True)

In [ ]:
class_weights = {
    0 : 1.02660468,
    1 : 9.40661861,
    2 : 1.00104606,
    3 : 0.56843877,
    4 : 0.84912748,
    5 : 1.29337298,
    6 : 0.82603942,
}

In [ ]:
H = model.fit(train_ds,steps_per_epoch = 1037, validation_data=val_ds,
            epochs=config_train["epochs"], verbose = 1, callbacks = [reduce_lr,model_checkpoint])


Epoch 1/100
1037/1037 [==============================] - ETA: 0s - loss: 2.0711 - accuracy: 0.2382
Epoch 1: val_accuracy improved from -inf to 0.31903, saving model to /content/drive/MyDrive/WorkSpace/AI_Research/Emotion/checkpoints/ferplus_vgg16_v1.hdf5
1037/1037 [==============================] - 426s 398ms/step - loss: 2.0711 - accuracy: 0.2382 - val_loss: 1.8423 - val_accuracy: 0.3190 - lr: 1.0000e-04
Epoch 2/100
1037/1037 [==============================] - ETA: 0s - loss: 1.7440 - accuracy: 0.3692
Epoch 2: val_accuracy improved from 0.31903 to 0.44943, saving model to /content/drive/MyDrive/WorkSpace/AI_Research/Emotion/checkpoints/ferplus_vgg16_v1.hdf5
1037/1037 [==============================] - 410s 395ms/step - loss: 1.7440 - accuracy: 0.3692 - val_loss: 1.5991 - val_accuracy: 0.4494 - lr: 1.0000e-04
Epoch 3/100
1037/1037 [==============================] - ETA: 0s - loss: 1.5749 - accuracy: 0.4431
Epoch 3: val_accuracy improved from 0.44943 to 0.51658, saving model to /content

In [ ]:
model = get_models.Pre_train_model(VGG16, img_conc, None)

# model.load_weights("/content/drive/MyDrive/WorkSpace/AI_Research/Emotion/checkpoints/FerPlusV11.hdf5")
model.compile(loss =config_train["loss"], optimizer =Adam(3e-4), metrics=[config_train["metric"]])
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=tf.math.exp(-0.1),
                              patience=3, min_lr=0 , verbose =1)

files = configfer["file_checkpoint"]
checkpoint_filepath = (f"{default_ws_dir}/checkpoints/{files}")

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose= 1,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=True, save_weights_only = True)

In [ ]:
H = model.fit(train_ds,steps_per_epoch = 1037, validation_data=val_ds, class_weight = class_weights,
            epochs=100, verbose = 1, callbacks = [reduce_lr,model_checkpoint])


Epoch 1/100


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-55edd6770d58>", line 2, in <module>
    epochs=100, verbose = 1, callbacks = [reduce_lr,model_checkpoint])
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 67, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/func_graph.py", line 1147, in autograph_handler
    raise e.ag_error_metadata.to_exception(e)
ValueError: in user code:

    File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, ar

ValueError: ignored

In [ ]:
# get_models.save_history("/content/drive/MyDrive/WorkSpace/AI_Research/Emotion/History/FerNew_V3.json", H)

In [ ]:

# def save_model(model,path)

# Evaluate

In [ ]:
model = get_models.Pre_train_model1(ResNet50, img_conc, None)
model.compile(loss ="categorical_crossentropy", optimizer =Adamax(3e-7), metrics=["accuracy"])
model.load_weights("/content/drive/MyDrive/WorkSpace/AI_Research/Emotion/checkpoints/FerCheckV6.hdf5")

In [ ]:
model.evaluate(test_ds)

In [ ]:
images , test_labels = predict.get_imgs_test(test_ds)

In [ ]:
get_models.Confusion_Matrix(model , images, test_labels, emotion , mode = True)

In [ ]:
model = get_models.Pre_train_model1(VGG16, img_conc, None)

# model.load_weights("/content/drive/MyDrive/WorkSpace/AI_Research/Emotion/checkpoints/FerCheckV6.hdf5")

model.compile(loss ="categorical_crossentropy", optimizer =Adamax(1e-4), metrics=["accuracy"])
# EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, min_lr=0)

checkpoint_filepath = "/content/drive/MyDrive/WorkSpace/AI_Research/Emotion/checkpoints/FerNew_vper.hdf5"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose= 1,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=True)


In [ ]:
H = model.fit(train_ds, validation_data=val_ds, steps_per_epoch = 952*2,
              epochs=80, verbose = 1, callbacks = [reduce_lr,model_checkpoint ])

In [ ]:
get_models.save_history("/content/drive/MyDrive/WorkSpace/AI_Research/Emotion/History/FerNew_V2.json", H)

In [10]:
model = get_models.Pre_train_model1(VGG16, img_conc, None)
model.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 concatenate (Concatenate)      (None, 224, 224, 3)  0           ['input_1[0][0]',                
                                                                  'input_1[0][0]',                
                                                                  'input_1[0][0]']                
                                                                                                  
 block1_conv1 (Conv2D)     

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Input, Conv2D, multiply, LocallyConnected2D, Lambda
from keras.models import Model
from keras.layers import BatchNormalization
# in_lay = Input((128,128,3))
base_pretrained_model = VGG16(input_shape = None, input_tensor = img_conc ,
                              include_top = False, weights = 'imagenet')
input = base_pretrained_model.input 
# base_pretrained_model.trainable = 

pt_depth = base_pretrained_model.layers[-1].output_shape[-1]
pt_features = base_pretrained_model.layers[-1].output

bn_features = BatchNormalization()(pt_features)

print(pt_features)
# here we do an attention mechanism to turn pixels in the GAP on an off
attn_layer = Conv2D(64, kernel_size = (1,1), padding = 'same', activation = 'relu')(bn_features)
attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
attn_layer = Conv2D(1, kernel_size = (1,1), padding = 'valid', activation = 'sigmoid')(attn_layer)

# fan it out to all of the channels
up_c2_w = np.ones((1, 1, 1, pt_depth))
up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', 
               activation = 'linear', use_bias = False, weights = [up_c2_w])
up_c2.trainable = False
attn_layer = up_c2(attn_layer)

mask_features = multiply([attn_layer, bn_features])
gap_features = GlobalAveragePooling2D()(mask_features)
gap_mask = GlobalAveragePooling2D()(attn_layer)
# to account for missing values from the attention model

gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
gap_dr = Dropout(0.5)(gap)

dr_steps = Dropout(0.25)(gap_dr)
dr_steps = (Dense(512, activation = 'relu',  kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                    bias_regularizer=regularizers.L2(1e-4),
                    activity_regularizer=regularizers.L2(1e-5)))(dr_steps)
dr_steps = Dropout(0.25)(dr_steps)
dr_steps = (Dense(128, activation = 'relu',  kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                    bias_regularizer=regularizers.L2(1e-4),
                    activity_regularizer=regularizers.L2(1e-5)))(dr_steps)


out_layer = Dense(7, activation = 'softmax')(dr_steps)

tb_model = Model(inputs = input, outputs = out_layer)
tb_model.summary()

In [ ]:
tb_model.compile(loss ="categorical_crossentropy", optimizer =Adamax(3e-4), metrics=["accuracy"])
# EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, min_lr=0)

checkpoint_filepath = "/content/drive/MyDrive/WorkSpace/AI_Research/Emotion/checkpoints/FerGAPV1.hdf5"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose= 1,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=True)

In [ ]:
H = tb_model.fit(train_ds, validation_data=val_ds, 
              epochs=80, verbose = 1, callbacks = [reduce_lr,model_checkpoint ])